In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
desc = pd.read_csv('data/snomed/sct2_Description_Full-en_US1000124_20180301.txt',sep='\t')
rel = pd.read_csv('data/snomed/sct2_Relationship_Full_US1000124_20180301.txt',sep='\t')

In [3]:
fi = open('inv_idx','rb')
inv_idx = pkl.load(fi)
keys = list(inv_idx.keys())

In [4]:
hkeys = []
hkey_map = {}
#There is a point implied between the third and fourth digit for all diagnosis codes other than the V codes.
#The decimal is implied for V codes between the second and third digit
for k in keys:
    if len(k) <= 3:
        continue
    if k[0] =='V':
        hkeys.append(k[0:3]+'.'+k[3:])
        hkey_map[k[0:3]+'.'+k[3:]] = k
    elif k[0] =='E':
        hkeys.append(k[0:4]+'.'+k[4:])
        hkey_map[k[0:4]+'.'+k[4:]] = k
    else:
        hkeys.append(k[0:3]+'.'+k[3:])
        hkey_map[k[0:3]+'.'+k[3:]] = k
        

In [5]:
map_icd_snomed = pd.read_csv('data/snomed/ICD9CM_SNOMED_MAP_1TO1_201712.txt',sep='\t')
map_icd_snomed = map_icd_snomed[['ICD_CODE','SNOMED_CID']]

multi_map_icd_snomed = pd.read_csv('data/snomed/ICD9CM_SNOMED_MAP_1TOM_201712.txt',sep='\t')
multi_map_icd_snomed = multi_map_icd_snomed[['ICD_CODE','SNOMED_CID']]


In [65]:
arr = map_icd_snomed.values
arr = dict([(str(key), str(value)) for value, key in arr])

In [66]:
snomed_v = {}
with open('data/snomed/vertexDictionary.out') as fv:
    for line in fv:
        line.replace('\n','')
        snomed, vertex = line.split('\t')
        snomed_v[int(vertex.replace('\n',''))] = str(snomed.replace('\n',''))

In [67]:
set_snomed_icd_keys = set(list(arr.keys()))


In [68]:
x = list(set_snomed_icd_keys)

In [69]:
#extracting only useful embeddings
def getEmbeddings():

    embeddings = {}
    with open('data/snomed/emb.out') as femb:
        meta = femb.readline()
        meta.replace('\n','')
        x, dim = meta.split(' ')

        for line in femb:
            line.replace('\n','')
            node_emb = line.split(' ')

            node = int(node_emb[0])
            node_emb = np.array(node_emb[1:], dtype=float)
            
            snomed = snomed_v[node]
            if snomed in set_snomed_icd_keys: 
                key = arr[snomed]
                embeddings[key] = node_emb

            # these embeddings were not given. so, we initialize them with 0
#     for vertex in range(0,284):
#         if vertex not in set(list(embeddings.keys())):
#             embeddings[vertex] = [0]*int(dimension)

    return embeddings



In [70]:
embeddings = getEmbeddings()

In [6]:
#use the icd9 codes to obtain the mapping from icd to snomed
icd_snomed = {}
snomed_icd = {}

# for icd, snomed in map_icd_snomed.values:
#     if icd in hkeys:
#         icd_snomed[icd] = snomed
#         snomed_icd[snomed] = icd

for icd, snomed in multi_map_icd_snomed.values:
    if icd in hkeys:
#         icd_snomed[icd] = snomed
        snomed_icd[snomed] = icd
        

In [7]:
skeys = list(snomed_icd.keys())
len(skeys)

40510

In [8]:
#extract all relations between any two icd codes available in MIMIC III
skeys = set(snomed_icd.keys())
mrel = []
for s, d, t in rel[['sourceId','destinationId','typeId']].values:
    if s in skeys and d in skeys:
        mrel.append([s,d,t])

In [98]:
# for k in icd_snomed.keys():
#     if k.startswith('E'):
#         print(k)

In [9]:
relset = set(np.array(mrel)[:,2])

In [10]:
for rs in relset:
    print(desc.loc[desc['conceptId']==rs][['conceptId','term']].values[0])

[42752001 'Due to']
[260908002 'Course']
[116680003 'Is a']
[363708005 'Temporally follows']
[116676008 'Associated morphology']
[308489006 'Pathological process']
[47429007 'Associated with']
[363705008 'Has definitional manifestation']
[255234002 'After']
[363714003 'Interprets']
[246090004 'Associated finding']
[363715002 'Associated etiologic finding']
[246075003 'Causative agent']


In [157]:
Assoc = rel.loc[rel['typeId']==47429007][['sourceId','destinationId']].values
adj =  [[0 for i in range(284)] for j in range(284)]
count = 15
for s,d in Assoc:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' ASSOCIATED WITH  '+snomed_icd[d] + ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]))
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' ASSOCIATED WITH  '+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break

        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
for i in range(0,284):
    for j in range(0,284):
        if adj[i][j] == 1:
            ed += 1

print(ed)


32


In [159]:
AssocE = rel.loc[rel['typeId']==246090004][['sourceId','destinationId']].values
adj =  [[0 for i in range(284)] for j in range(284)]
count = 15
for s,d in AssocE:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' ASSOCIATED WITH  '+snomed_icd[d] + ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]))
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' ASSOCIATED WITH  '+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break

        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
for i in range(0,284):
    for j in range(0,284):
        if adj[i][j] == 1:
            ed += 1

print(ed)


52


In [152]:
Manif = rel.loc[rel['typeId']==363705008][['sourceId','destinationId']].values
count = 15
adj =  [[0 for i in range(284)] for j in range(284)]

for s,d in Manif:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' has definitional Manifestation  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' has definitional Manifestation  '.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break
        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
for i in range(0,284):
    for j in range(0,284):
        if adj[i][j] == 1:
            ed += 1

print(ed)

16


In [153]:
After = rel.loc[rel['typeId']==255234002][['sourceId','destinationId']].values
count = 15
adj =  [[0 for i in range(284)] for j in range(284)]
for s,d in After:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' after  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' after  '.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break
        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
for i in range(0,284):
    for j in range(0,284):
        if adj[i][j] == 1:
            ed += 1

print(ed)

32


In [12]:
DueTo = rel.loc[rel['typeId']==42752001][['sourceId','destinationId']].values
count = 20
adj =  [[0 for i in range(284)] for j in range(284)]
edgset = set([])

for s,d in DueTo:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' Due To  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' Due To  '.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break
#         x, y = snomed_icd[s].split('.')[0],snomed_icd[d].split('.')[0]
#         edgset = edgset | set([(x,y)])
        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1


# print((edgset))
ed = 0
adjlist = {}
for i in range(0,284):
    adjlist[i]= []
    for j in range(0,284):
        if adj[i][j] == 1 and not i == j:
            ed += 1
            adjlist[i].append(j)

print(ed)
relf = open('dueTo.rel','wb')
pkl.dump(adjlist, relf)
relf.close()

206


In [13]:
print(adj[69][57])

1


In [14]:
IsA = rel.loc[rel['typeId']==116680003][['sourceId','destinationId']].values
count = 20
adj =  [[0 for i in range(284)] for j in range(284)]
for s,d in IsA:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' Is A  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' Is A'.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break
        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
adjlist = {}
for i in range(0,284):
    adjlist[i]= []
    for j in range(0,284):
        if adj[i][j] == 1 and not i == j:
            ed += 1
            adjlist[i].append(j)
print(ed)
relf = open('isA.rel','wb')
pkl.dump(adjlist, relf)
relf.close()


3135


In [145]:
Follows = rel.loc[rel['typeId']==363708005][['sourceId','destinationId']].values
count = 20
adj =  [[0 for i in range(284)] for j in range(284)]
for s,d in Follows:
    if s in skeys and d in skeys:
#         print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' temporally follows  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
#         print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' temporally follows '.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
#         print('\n')
#         count -= 1
        
#         if count == 0:
#             break
        adj[inv_idx[(hkey_map[snomed_icd[s]])]][inv_idx[(hkey_map[snomed_icd[d]])]] = 1

    
ed = 0
for i in range(0,284):
    for j in range(0,284):
        if adj[i][j] == 1:
            ed += 1
print(ed)

13


In [139]:
CA = rel.loc[rel['typeId']==246075003][['sourceId','destinationId']].values
count = 20
for s,d in CA:
    if not( s in skeys and d in skeys):
        #print('SOURCE '+snomed_icd[s]+ 'group : ' +str(inv_idx[(hkey_map[snomed_icd[s]])])+ ' temporally follows  '.upper() +snomed_icd[d]+ ' group : ' +str(inv_idx[(hkey_map[snomed_icd[d]])]) )
        print('SOURCE '+desc.loc[desc['conceptId']==s]['term'].values[0]+' causative agent '.upper()+desc.loc[desc['conceptId']==d]['term'].values[0])
        print('\n')
        count -= 1
        
        if count == 0:
            break

SOURCE Bacterial septicemia CAUSATIVE AGENT Bacterium


SOURCE Bacterial septicemia CAUSATIVE AGENT Bacterium


SOURCE Uncomplicated sedative, hypnotic AND/OR anxiolytic withdrawal CAUSATIVE AGENT Drug


SOURCE Uncomplicated sedative, hypnotic AND/OR anxiolytic withdrawal CAUSATIVE AGENT Drug


SOURCE Uncomplicated sedative, hypnotic AND/OR anxiolytic withdrawal CAUSATIVE AGENT Psychoactive substance of abuse - non-pharmaceutical


SOURCE Uncomplicated sedative, hypnotic AND/OR anxiolytic withdrawal CAUSATIVE AGENT Psychoactive substance of abuse - non-pharmaceutical


SOURCE Spontaneous abortion with amniotic fluid embolism CAUSATIVE AGENT Amniotic fluid


SOURCE Poisoning by quinoline AND/OR hydroxyquinoline derivative CAUSATIVE AGENT Antimalarial agent


SOURCE Poisoning by quinoline AND/OR hydroxyquinoline derivative CAUSATIVE AGENT Antimalarial agent


SOURCE Poisoning by quinoline AND/OR hydroxyquinoline derivative CAUSATIVE AGENT Narcotic drug


SOURCE Poisoning by quinoline AND